In [22]:
import pandas as pd
import joblib
from pyecharts.charts import *
import pyecharts.options as opts
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.naive_bayes import BernoulliNB,GaussianNB,MultinomialNB
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import os
pd.set_option('display.max_columns', None)

In [23]:
df_data_path = "Train.csv"
df_data = pd.read_csv(df_data_path,index_col='ID')

dfV_data_path = "Validate.csv"
dfV_data = pd.read_csv(dfV_data_path,index_col='ID')

dfT_data_path = "Test.csv"
dfT_data = pd.read_csv(dfT_data_path,index_col='ID')

In [24]:
# 2.1 查看数据内容
df_data.head(5)

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target)
ID,,,,,,,,,,
0,1,1.0,36,NaN,Artist,9.0,Low,2.0,Cat_6,B
1,0,1.0,49,1.0,Artist,1.0,Low,1.0,Cat_6,D
2,0,NaN,25,0.0,Homemaker,8.0,Low,1.0,Cat_3,A
3,1,0.0,77,1.0,Lawyer,0.0,High,2.0,Cat_4,D
4,1,0.0,39,1.0,Entertainment,1.0,Average,4.0,Cat_3,D


In [25]:
# 2.2 查看各列数据类型信息
df_data.dtypes

Married                int64
Gender               float64
Age                    int64
Graduate             float64
Profession            object
Years_of_Working     float64
Spending_Score        object
Family_Members       float64
Category              object
Class(Target)         object
dtype: object

In [26]:
df_data.isna().sum()

Married                0
Gender               130
Age                    0
Graduate              77
Profession           112
Years_of_Working     742
Spending_Score         0
Family_Members       310
Category              72
Class(Target)          0
dtype: int64

In [27]:
dfV_data

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target)
ID,,,,,,,,,,
7487,0,0.0,36,1.0,Doctor,13.0,Low,2.0,Cat_6,B
7488,1,0.0,43,0.0,Executive,0.0,High,4.0,Cat_6,B
7489,0,0.0,53,1.0,Engineer,0.0,Average,4.0,Cat_6,A
7490,0,1.0,51,1.0,Lawyer,2.0,Low,1.0,Cat_6,B
7491,1,1.0,25,1.0,Marketing,NaN,Low,5.0,Cat_3,D
...,...,...,...,...,...,...,...,...,...,...
8551,0,0.0,38,1.0,Marketing,9.0,Low,3.0,Cat_6,A
8552,1,0.0,47,0.0,Entertainment,0.0,Low,2.0,Cat_6,B
8553,0,1.0,22,0.0,Healthcare,0.0,Low,5.0,Cat_6,A


In [28]:
dfT_data

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target)
ID,,,,,,,,,,
8556,1,1.0,25,0.0,Healthcare,0.0,Low,5.0,Cat_6,NaN
8557,1,1.0,27,0.0,Healthcare,1.0,Low,3.0,NaN,NaN
8558,0,0.0,65,1.0,Lawyer,1.0,Low,2.0,Cat_3,NaN
8559,1,0.0,47,1.0,Executive,3.0,High,3.0,Cat_6,NaN
8560,0,1.0,31,0.0,Doctor,1.0,Low,5.0,Cat_3,NaN
...,...,...,...,...,...,...,...,...,...,...
10690,1,1.0,18,0.0,Healthcare,1.0,Low,4.0,Cat_6,NaN
10691,0,1.0,25,0.0,Healthcare,0.0,Low,4.0,Cat_2,NaN
10692,0,1.0,21,0.0,Healthcare,8.0,Low,7.0,Cat_2,NaN


In [29]:
df_data = df_data.dropna()
df_data
# dfV_data = dfV_data.dropna()
# dfV_data

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target)
ID,,,,,,,,,,
1,0,1.0,49,1.0,Artist,1.0,Low,1.0,Cat_6,D
3,1,0.0,77,1.0,Lawyer,0.0,High,2.0,Cat_4,D
4,1,0.0,39,1.0,Entertainment,1.0,Average,4.0,Cat_3,D
6,0,1.0,40,1.0,Artist,8.0,Low,2.0,Cat_6,B
7,0,1.0,37,0.0,Homemaker,8.0,Low,1.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...
7481,1,1.0,27,1.0,Healthcare,0.0,Low,2.0,Cat_6,D
7483,0,1.0,37,1.0,Healthcare,1.0,Low,2.0,Cat_6,C
7484,0,0.0,66,0.0,Doctor,9.0,Average,2.0,Cat_6,C


In [30]:
# dfT_data = dfT_data.dropna(subset=['Married','Gender','Age','Graduate','Profession','Years_of_Working ','Spending_Score','Family_Members','Category'])
# dfT_data

mean = dfT_data.mean()
dfT_data = dfT_data.fillna(mean)

dfT_data = dfT_data.fillna(method='backfill', axis=0, inplace=False)


mean = dfV_data.mean()
dfV_data = dfV_data.fillna(mean)

dfV_data = dfV_data.fillna(method='backfill', axis=0, inplace=False)


mean = df_data.mean()
df_data = df_data.fillna(mean)

df_data = df_data.fillna(method='backfill', axis=0, inplace=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_25272\1375837751.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean = dfT_data.mean()
C:\Users\lenovo\AppData\Local\Temp\ipykernel_25272\1375837751.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean = dfV_data.mean()
C:\Users\lenovo\AppData\Local\Temp\ipykernel_25272\1375837751.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean = df_data.mean()


In [31]:
df_data.isna().sum()

Married              0
Gender               0
Age                  0
Graduate             0
Profession           0
Years_of_Working     0
Spending_Score       0
Family_Members       0
Category             0
Class(Target)        0
dtype: int64

In [32]:
dfV_data.isna().sum()

Married              0
Gender               0
Age                  0
Graduate             0
Profession           0
Years_of_Working     0
Spending_Score       0
Family_Members       0
Category             0
Class(Target)        0
dtype: int64

In [33]:
dfT_data.isna().sum()

Married                 0
Gender                  0
Age                     0
Graduate                0
Profession              0
Years_of_Working        0
Spending_Score          0
Family_Members          0
Category                0
Class(Target)        2139
dtype: int64

In [34]:
# 2.3 查看数据中数值变量的统计信息
df_data.describe()

,Married,Gender,Age,Graduate,Years_of_Working,Family_Members
count,6212.000000,6212.000000,6212.000000,6212.000000,6212.000000,6212.000000
mean,0.553928,0.412910,43.433516,0.639730,2.578075,2.828236
std,0.497123,0.492397,16.511192,0.480117,3.376668,1.528020
min,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,30.000000,0.000000,0.000000,2.000000
50%,1.000000,0.000000,41.000000,1.000000,1.000000,2.000000
75%,1.000000,1.000000,53.000000,1.000000,4.000000,4.000000
max,1.000000,1.000000,89.000000,1.000000,14.000000,9.000000


In [35]:
# 2.5 查看重复数据状况
df_data.duplicated().sum()  # 可以看到重复数据还是挺多的，但原数据已有唯一标识id，考虑到应该是不同时间的同一航线信息，故不做去重处理。

320

In [36]:
np.max(dfV_data['Age'])

89

In [37]:
df_data = df_data.join(pd.get_dummies(pd.cut(df_data['Age'], [0, 18, 60, 89], labels=['Age1','Age2','Age3'])))

df_data['Profession'] = df_data['Profession'].astype(str)
temp_df = df_data['Profession'].unique()
label_encoder = LabelEncoder().fit(temp_df)
temp_df_label = label_encoder.transform(temp_df)
df_data['Profession'] = label_encoder.transform(df_data['Profession'])

temp_dfV = dfV_data['Profession'].unique()
temp_dfV_label = label_encoder.transform(temp_dfV)
dfV_data['Profession'] = label_encoder.transform(dfV_data['Profession'])

temp_dfT = dfT_data['Profession'].unique()
temp_dfT_label = label_encoder.transform(temp_dfT)
dfT_data['Profession'] = label_encoder.transform(dfT_data['Profession'])


df_data['Spending_Score'] = df_data['Spending_Score'].astype(str)
temp_df = df_data['Spending_Score'].unique()
label_encoder = LabelEncoder().fit(temp_df)
temp_df_label = label_encoder.transform(temp_df)
df_data['Spending_Score'] = label_encoder.transform(df_data['Spending_Score'])

temp_dfV = dfV_data['Spending_Score'].unique()
temp_dfV_label = label_encoder.transform(temp_dfV)
dfV_data['Spending_Score'] = label_encoder.transform(dfV_data['Spending_Score'])

temp_dfT = dfT_data['Spending_Score'].unique()
temp_dfT_label = label_encoder.transform(temp_dfT)
dfT_data['Spending_Score'] = label_encoder.transform(dfT_data['Spending_Score'])


df_data['Category'] = df_data['Category'].astype(str)
temp_df = df_data['Category'].unique()
label_encoder = LabelEncoder().fit(temp_df)
temp_df_label = label_encoder.transform(temp_df)
df_data['Category'] = label_encoder.transform(df_data['Category'])

temp_dfV = dfV_data['Category'].unique()
temp_dfV_label = label_encoder.transform(temp_dfV)
dfV_data['Category'] = label_encoder.transform(dfV_data['Category'])

temp_dfT = dfT_data['Category'].unique()
temp_dfT_label = label_encoder.transform(temp_dfT)
dfT_data['Category'] = label_encoder.transform(dfT_data['Category'])


df_data['Class(Target)'] = df_data['Class(Target)'].astype(str)
temp_df = df_data['Class(Target)'].unique()
label_encoder = LabelEncoder().fit(temp_df)
temp_df_label = label_encoder.transform(temp_df)
df_data['Class(Target)'] = label_encoder.transform(df_data['Class(Target)'])

temp_dfV = dfV_data['Class(Target)'].unique()
temp_dfV_label = label_encoder.transform(temp_dfV)
dfV_data['Class(Target)'] = label_encoder.transform(dfV_data['Class(Target)'])



X_train_d = df_data.loc[:,['Married','Gender','Age','Graduate','Profession','Years_of_Working ','Spending_Score','Family_Members','Category']]
y_train_d = df_data.loc[:,'Class(Target)']

X_test_d = dfV_data.loc[:,['Married','Gender','Age','Graduate','Profession','Years_of_Working ','Spending_Score','Family_Members','Category']]
y_test_d = dfV_data.loc[:,'Class(Target)']


# X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size=0, random_state=2)

In [38]:
df_data

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target),Age1,Age2,Age3
ID,,,,,,,,,,,,,
1,0,1.0,49,1.0,0,1.0,2,1.0,5,3,0,1,0
3,1,0.0,77,1.0,7,0.0,1,2.0,3,3,0,0,1
4,1,0.0,39,1.0,3,1.0,0,4.0,2,3,0,1,0
6,0,1.0,40,1.0,0,8.0,2,2.0,5,1,0,1,0
7,0,1.0,37,0.0,6,8.0,2,1.0,5,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7481,1,1.0,27,1.0,5,0.0,2,2.0,5,3,0,1,0
7483,0,1.0,37,1.0,5,1.0,2,2.0,5,2,0,1,0
7484,0,0.0,66,0.0,1,9.0,0,2.0,5,2,0,0,1


In [39]:
# from sklearn.model_selection import KFold
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import f1_score


# def myFeval(preds, dtrain): # 定义你自己的评价标准
#     labels = dtrain.get_label()
#     return Score, f1_score(labels , preds, average='macro')  

# # 读取数据
# validata = dfV_data
# valilabel = pd.DataFrame(validata['Class(Target)'])

# modelres1 = []
# kf = KFold(n_splits=10, random_state=2022, shuffle=True) # 10折交叉验证
# for index, (train_index, test_index) in enumerate(kf.split(validata, valilabel)):
# 	# 通过train_index 和 test_index 分别取原始数据中对应的行作为训练集以及测试集
#     X_train, X_test = validata.iloc[train_index][:].values, validata.iloc[test_index][:].values
#     y_train, y_test = valilabel.iloc[train_index][:].values, valilabel.iloc[test_index][:].values

#     Model1 = xgb.XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=150)
#     Model1.fit(X_train, y_train.ravel())

#     t = Model1.predict(X_test).tolist()
#     modelres1.append(myFeval(t, y_test).item()) # 单次交叉验证的得分结果


In [41]:
# 建立模型应用效果记录函数
model_details = []
def modle_pro(modle_):
    # 创建保存模型信息的list
    modle_d_list = []
    # 初始化模型
    modle = modle_
    # 记录模型训练运行时间
    old_time = time.time()
    modle.fit(X_train_d,y_train_d)
    current_time = time.time()
    use_time = round(current_time-old_time,4)
    # 模型预测
    pre = modle.predict(X_test_d)
    pre = pre.astype(np.int64)
    # 采用accuracy_score作为评估指标
    precision = round(accuracy_score(pre,y_test_d),4)
    modle_d_list.append(str(modle_))
    modle_d_list.append(use_time)
    modle_d_list.append(precision)
    model_details.append(modle_d_list)
# 高斯朴素贝叶斯
gnb = GaussianNB()
modle_pro(gnb)

# 伯努利朴素贝叶斯
bnb = BernoulliNB()
modle_pro(bnb)
# 多项式朴素贝叶斯
mnb = MultinomialNB()
modle_pro(mnb)
# K近邻
knn = KNeighborsClassifier(n_neighbors=3)
modle_pro(knn)
# 线性回归
lr = LinearRegression()
modle_pro(lr)
# 逻辑回归
lgr = LogisticRegression()
modle_pro(lgr)
# 决策树-使用信息熵
dtc_e = DecisionTreeClassifier(criterion='entropy')
modle_pro(dtc_e)
# 决策树-使用基尼系数
dtc_g = DecisionTreeClassifier(criterion='gini',max_depth=20)
modle_pro(dtc_g)
# # 随机森林
rfc = RandomForestClassifier()
modle_pro(rfc)

clf = GradientBoostingClassifier(n_estimators=24)
modle_pro(clf)

svm = SVC(kernel='linear',C=1.0,random_state=1)
modle_pro(svm)


gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
modle_pro(gbm)


xgbc = XGBClassifier(learning_rate=0.01,
                      n_estimators=150,           # 树的个数-10棵树建立xgboost
                      max_depth=5,               # 树的深度
                      min_child_weight = 3,      # 叶子节点最小权重
                      gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,               # 所有样本建立决策树
                      colsample_btree=0.8,         # 所有特征建立决策树
#                       scale_pos_weight=1,        # 解决样本个数不平衡的问题
#                       random_state=27,           # 随机数
#                       slient = 0
                     objective= 'reg:squarederror',
                   reg_alpha= 0,
                   reg_lambda= 1,
                   nthread=4,
                   scale_pos_weight=1,
                   seed=27
                    )
modle_pro(xgbc)


df = pd.DataFrame(model_details, columns=['modle_name', 'use_time', 'accuracy_score'])
df.sort_values('accuracy_score',ascending=False)

C:\Users\lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\lenovo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

[11:45:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "colsample_btree", "scale_pos_weight" } are not used.



,modle_name,use_time,accuracy_score
9,GradientBoostingClassifier(n_estimators=24),0.3202,0.4846
12,"XGBClassifier(base_score=None, booster=None, c...",0.7618,0.4677
10,"SVC(kernel='linear', random_state=1)",4.9145,0.4462
5,LogisticRegression(),0.1752,0.4434
0,GaussianNB(),0.0030,0.4210
8,RandomForestClassifier(),0.4074,0.4191
1,BernoulliNB(),0.0040,0.4153
2,MultinomialNB(),0.0010,0.4079
3,KNeighborsClassifier(n_neighbors=3),0.0128,0.3854
6,DecisionTreeClassifier(criterion='entropy'),0.0169,0.3751


In [42]:
from sklearn.model_selection import GridSearchCV
#Need to research
#research_one: n_epoch
#research_one: max_depth
param_test1 = {
    'min_child_weight': [1, 2, 3],
    'max_depth':[2, 3, 4, 5, 6, 7]
    }

xgbc_res = GridSearchCV(estimator = xgbc, 
                       param_grid = param_test1, 
                       n_jobs=4, 
                       cv=5)

xgbc_res.fit(X_train_d, y_train_d)

[11:45:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "colsample_btree", "scale_pos_weight" } are not used.



GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_btree=0.8,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=0.0, gpu_id=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.01, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                       

In [43]:
print('max_depth_min_child_weight')
print('gsearch1.grid_scores_', xgbc_res.cv_results_)
print('gsearch1.best_params_', xgbc_res.best_params_)
print('gsearch1.best_score_', xgbc_res.best_score_)


max_depth_min_child_weight
gsearch1.grid_scores_ {'mean_fit_time': array([0.51284957, 0.4684134 , 0.45829859, 0.61593962, 0.69835825,
       0.63796425, 0.81018357, 0.81600385, 0.88480043, 1.02603259,
       1.05524621, 1.05876346, 1.26950021, 1.27569623, 1.35562453,
       1.69131041, 1.58670473, 1.65554738]), 'std_fit_time': array([0.01941238, 0.00543644, 0.00482784, 0.00306018, 0.04361459,
       0.02885887, 0.01756536, 0.02108677, 0.03189815, 0.01463755,
       0.02801762, 0.01634737, 0.03599055, 0.05939939, 0.03192348,
       0.04387899, 0.09696361, 0.17667763]), 'mean_score_time': array([0.00480113, 0.00443625, 0.004601  , 0.00600157, 0.00580134,
       0.00558071, 0.00700173, 0.00758939, 0.0068017 , 0.00800171,
       0.00860205, 0.00824971, 0.01000237, 0.01040225, 0.01020246,
       0.0126029 , 0.01278715, 0.01100254]), 'std_score_time': array([4.00114074e-04, 4.67193650e-04, 4.89784675e-04, 2.43140197e-07,
       3.99899550e-04, 4.83674275e-04, 2.43140197e-07, 8.10033870e-04,


In [44]:
X_test1_d = dfT_data.loc[:,['Married','Gender','Age','Graduate','Profession','Years_of_Working ','Spending_Score','Family_Members','Category']]
y_test1_d = dfT_data.loc[:,'Class(Target)']
pre = dtc_g.predict(X_test1_d)
pre=label_encoder.inverse_transform(pre)

In [45]:
joblib.dump(dtc_g, "./dtc_g.pkl")

['./dtc_g.pkl']

In [46]:
pre

array(['A', 'C', 'D', ..., 'A', 'A', 'B'], dtype=object)

In [47]:
dfT_data['Class(Target)']=pre

In [48]:
len(pre)

2139

In [49]:
dfT_data.head()

,Married,Gender,Age,Graduate,Profession,Years_of_Working,Spending_Score,Family_Members,Category,Class(Target)
ID,,,,,,,,,,
8556,1,1.0,25,0.0,5,0.0,2,5.0,5,A
8557,1,1.0,27,0.0,5,1.0,2,3.0,2,C
8558,0,0.0,65,1.0,7,1.0,2,2.0,2,D
8559,1,0.0,47,1.0,4,3.0,1,3.0,5,D
8560,0,1.0,31,0.0,1,1.0,2,5.0,2,A


In [50]:
dfT_data.to_csv('test1.csv') 